# Deploying functions 

In [391]:
#Reference
#https://docs.openfaas.com/tutorials/kubernetes-hpa/

In [392]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f aeneas.yml --annotation topic="aeneas" --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=15 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: aeneas.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/aeneas



In [393]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f scaling-aeneas-tocloud.yml --annotation topic="tocloud" --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: scaling-aeneas-tocloud.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/scaling-aeneas-tocloud



In [394]:
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f getobject.yml  --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: getobject.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/getobject



In [395]:
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f pubrabbitmq.yml  --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: pubrabbitmq.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/pubrabbitmq



In [396]:
!ssh ubuntu@172.17.141.197 "faas describe aeneas"

Name:               aeneas
Status:             Ready
Replicas:           1
Available Replicas: 1
Invocations:        38552
Image:              shivupoojar/pythonaeneasrabbitmq:latest
Function Process:   python index.py
URL:                http://127.0.0.1:8080/function/aeneas
Async URL:          http://127.0.0.1:8080/async-function/aeneas
Labels:
 com.openfaas.scale.target-proportion: 0.90
 com.openfaas.scale.type: rps
 faas_function: aeneas
 uid: 166444831
 com.openfaas.scale.max: 15
 com.openfaas.scale.min: 1
Annotations:
 topic: aeneas
 prometheus.io.scrape: false
Requests:  CPU: 1
 Memory: 512Mi


# Deploying Rabbitmq Connector 

In [397]:
# Deploying connector configurartion
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply -f cf.yaml"

configmap/rabbitmq-connector-configmap unchanged


In [398]:
# Deploy connector
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl delete  -f configmap.yaml"

deployment.apps "rabbitmq-connector" deleted


In [399]:
# Deploy connector
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply  -f configmap.yaml"

deployment.apps/rabbitmq-connector created


In [400]:
!ssh ubuntu@172.17.141.197 "sudo kubectl get po -n openfaas" 

NAME                                  READY   STATUS              RESTARTS         AGE
nats-5b77986b8-48p2x                  1/1     Running             4 (5d20h ago)    41d
queue-worker-7d8668444f-tmq7b         1/1     Running             5 (5d20h ago)    27d
alertmanager-6cd7bd5c64-hrcrt         1/1     Running             4 (5d20h ago)    27d
basic-auth-plugin-78b958969c-tnpck    1/1     Running             4 (5d20h ago)    27d
gateway-bf4f5d6f7-4q7kn               2/2     Running             10 (4d15h ago)   27d
prometheus-7678c644d6-k28wr           1/1     Running             0                2m22s
rabbitmq-connector-64cbd548f5-vtw26   0/1     ContainerCreating   0                2s


# Experiments 

In [401]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

In [402]:
scenario = "rps"
user=10
iteration=0.05
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:31022/'
time_locust=120


In [403]:
!rm input_data.csv
!touch input_data.csv
!echo "intime" > input_data.csv

In [404]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    client.remove_object(bucket_name, obj.object_name)

In [405]:
!locust --headless -f testingrabbitlocust.py --host='127.0.0.1' --skip-log -t 120s  --users 10 --spawn-rate 0.05

In [406]:
time.sleep(60)

In [407]:
df= pd.read_csv('input_data.csv')
print(df)

                          intime
0   '2023-01-17 10:37:04.160635'
1   '2023-01-17 10:37:16.826834'
2   '2023-01-17 10:37:24.160216'
3   '2023-01-17 10:37:25.476715'
4   '2023-01-17 10:37:33.568408'
5   '2023-01-17 10:37:33.930749'
6   '2023-01-17 10:37:42.427527'
7   '2023-01-17 10:37:44.150325'
8   '2023-01-17 10:37:45.934681'
9   '2023-01-17 10:37:53.119861'
10  '2023-01-17 10:37:54.398629'
11  '2023-01-17 10:37:57.745100'
12  '2023-01-17 10:38:01.716402'
13  '2023-01-17 10:38:03.177694'
14  '2023-01-17 10:38:04.150174'
15  '2023-01-17 10:38:08.933523'
16  '2023-01-17 10:38:10.816638'
17  '2023-01-17 10:38:12.290380'
18  '2023-01-17 10:38:14.782355'
19  '2023-01-17 10:38:19.495904'
20  '2023-01-17 10:38:20.371819'
21  '2023-01-17 10:38:24.150913'
22  '2023-01-17 10:38:25.549842'
23  '2023-01-17 10:38:26.001082'
24  '2023-01-17 10:38:27.331556'
25  '2023-01-17 10:38:29.373552'
26  '2023-01-17 10:38:34.293806'
27  '2023-01-17 10:38:34.474609'
28  '2023-01-17 10:38:34.595200'
29  '2023-

In [408]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)#
    client.remove_object(bucket_name, obj.object_name)

In [409]:
df['outtime']= outtime
df['outtime']= pd.to_datetime(df['outtime'],errors='coerce').dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'],errors='coerce')
df =df.sort_values(by="outtime")
display(df)
print(df['outtime'].iloc[-1]-df['intime'].iloc[0])

,intime,outtime
0,2023-01-17 10:37:04.160635,2023-01-17 10:37:09.191
1,2023-01-17 10:37:16.826834,2023-01-17 10:37:21.175
2,2023-01-17 10:37:24.160216,2023-01-17 10:37:29.542
3,2023-01-17 10:37:25.476715,2023-01-17 10:37:31.946
4,2023-01-17 10:37:33.568408,2023-01-17 10:37:40.332
5,2023-01-17 10:37:33.930749,2023-01-17 10:37:40.465
6,2023-01-17 10:37:42.427527,2023-01-17 10:37:47.775
7,2023-01-17 10:37:44.150325,2023-01-17 10:37:51.061
8,2023-01-17 10:37:45.934681,2023-01-17 10:37:53.613
9,2023-01-17 10:37:53.119861,2023-01-17 10:37:58.765


0 days 00:02:03.298365


In [410]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))

In [411]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn  -n openfaas -c gateway  | grep /function/aeneas | grep 200 | cut -c 80-83 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
cmd3="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/getobject | grep POST | cut -c 71-76  | tail -"+str(len(outtime))
cmd4="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/pubrabbitmq | grep POST | cut -c 73-78 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
#df['aeneas']=(pd.to_timedelta(df['aeneas'].astype(float), unit='s'))

df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]
#df['tocloud']=(pd.to_timedelta(df['tocloud'].astype(float), unit='s'))

#df['getobject']=(get_function_execution_time(cmd3)).split("\n")[:-1]
df['pubrabbitmq']=((get_function_execution_time(cmd4)).split("\n")[:-1])
#df['pubrabbitmq']=(pd.to_timedelta(df['pubrabbitmq'].astype(float), unit='s'))

df

,intime,outtime,aeneas,tocloud,pubrabbitmq
0,2023-01-17 10:37:04.160635,2023-01-17 10:37:09.191,4.85,0.0483,0.0396
1,2023-01-17 10:37:16.826834,2023-01-17 10:37:21.175,4.25,0.0436,0.0351
2,2023-01-17 10:37:24.160216,2023-01-17 10:37:29.542,5.29,0.1172,0.0341
3,2023-01-17 10:37:25.476715,2023-01-17 10:37:31.946,6.38,0.0416,0.0354
4,2023-01-17 10:37:33.568408,2023-01-17 10:37:40.332,6.70,0.0414,0.0434
5,2023-01-17 10:37:33.930749,2023-01-17 10:37:40.465,6.43,0.0385,0.0302
6,2023-01-17 10:37:42.427527,2023-01-17 10:37:47.775,5.25,0.0667,0.0389
7,2023-01-17 10:37:44.150325,2023-01-17 10:37:51.061,6.81,0.0451,0.0364
8,2023-01-17 10:37:45.934681,2023-01-17 10:37:53.613,7.59,0.0431,0.0347
9,2023-01-17 10:37:53.119861,2023-01-17 10:37:58.765,5.53,0.0416,0.0446


In [412]:
#df =df.sort_values(by="outtime")
df['TPT']=(df['outtime']-df['intime'])
df['TPT']=df['TPT'].values.astype('datetime64[ns]')
df['TPT']= pd.to_datetime(df['TPT']).dt.strftime('%S.%f')
df['TPT']=df['TPT'].astype(float)
df['PT']= df['TPT'].round(decimals = 0)
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
#df['MQ time']= ((df['TPT']).astype(float) - df['FET'])
df["scenario"]= ""+str(user)+"_"+str(iteration)+"_"+scenario
display(df)
df.to_csv(str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')

,intime,outtime,aeneas,tocloud,pubrabbitmq,TPT,PT,FET,scenario
0,2023-01-17 10:37:04.160635,2023-01-17 10:37:09.191,4.85,0.0483,0.0396,5.030365,5.0,4.8983,10_0.05_rps
1,2023-01-17 10:37:16.826834,2023-01-17 10:37:21.175,4.25,0.0436,0.0351,4.348166,4.0,4.2936,10_0.05_rps
2,2023-01-17 10:37:24.160216,2023-01-17 10:37:29.542,5.29,0.1172,0.0341,5.381784,5.0,5.4072,10_0.05_rps
3,2023-01-17 10:37:25.476715,2023-01-17 10:37:31.946,6.38,0.0416,0.0354,6.469285,6.0,6.4216,10_0.05_rps
4,2023-01-17 10:37:33.568408,2023-01-17 10:37:40.332,6.70,0.0414,0.0434,6.763592,7.0,6.7414,10_0.05_rps
5,2023-01-17 10:37:33.930749,2023-01-17 10:37:40.465,6.43,0.0385,0.0302,6.534251,7.0,6.4685,10_0.05_rps
6,2023-01-17 10:37:42.427527,2023-01-17 10:37:47.775,5.25,0.0667,0.0389,5.347473,5.0,5.3167,10_0.05_rps
7,2023-01-17 10:37:44.150325,2023-01-17 10:37:51.061,6.81,0.0451,0.0364,6.910675,7.0,6.8551,10_0.05_rps
8,2023-01-17 10:37:45.934681,2023-01-17 10:37:53.613,7.59,0.0431,0.0347,7.678319,8.0,7.6331,10_0.05_rps
9,2023-01-17 10:37:53.119861,2023-01-17 10:37:58.765,5.53,0.0416,0.0446,5.645139,6.0,5.5716,10_0.05_rps


In [413]:
pre_url = PROMETHEUS + '/api/v1/query_range?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time_interval = (time.mktime(timestamp.timetuple()))
user=df.shape[0]
end=(time.mktime(timestamp.timetuple()))
start=pd.to_datetime(df['intime'].iloc[0], format='%Y-%m-%d %H:%M:%S')
start=(time.mktime(start.timetuple()))
print(start,end)

1673951824.0 1673951947.0


In [414]:
interval = (df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()
interval= round(interval)

In [415]:
interval= round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) if round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) > 60  else 60
print(interval)

123


In [416]:
import copy 
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    #data2=res.get('data').get('result')[0].get('values')
    data_dict={}
    metric_list = []
    # print(data['data']['result']['values'])
    # exit()
    for i in res['data']['result']:
        for j in i['values']:
            data_dict = copy.deepcopy(i['metric'])
            data_dict['time'] = j[0]
            data_dict['value'] = j[1]
            metric_list.append(data_dict)
    df_metric = pd.DataFrame(metric_list)
    return df_metric

In [417]:
user=df.shape[0]
metrics = {'workload_type':""+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration),'workload_size':user,'Workload TPT':(df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean()}

promo_metrics = pd.DataFrame(columns=['cpu_used','cpu_requested'])

In [418]:

expr_namespace_cpu_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['value']

promo_metrics['time']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['time']
expr_namespace_cpu_used = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)['value']

expr_namespace_pod_count = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['pod_count']=getdataprometheus(pre_url+expr_namespace_pod_count)['value']

expr_namespace_memory_used = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_used)['value']

expr_namespace_memory_requested = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_requested)['value']

promo_metrics['time']= pd.to_datetime(promo_metrics['time'],unit='s')
seconds = []
for i in range(promo_metrics.shape[0]): 
    seconds.append(i*10)
promo_metrics['seconds'] = seconds

throughput_qurty = 'sum (rate(gateway_function_invocation_total{code="200"}[10s]))'
promo_metrics['throughput']= getdataprometheus('http://172.17.141.197:31376/api/v1/query_range?query=sum (rate(gateway_function_invocation_total{code="200"}[20s]))&start='+str(start)+'&end='+str(end)+'&step=10s')['value']
promo_metrics.to_csv("keda_results/promo_metrics"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv',index=False)


# #expr_namespace_cpu_requested = 'sum(node_namespace_pod_container:container_cpu_usage_seconds_total:sum_irate{namespace="openfaas-fn"})&time='+str(time_interval)
# expr_namespace_cpu_used='sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)
# expr_namespace_cpu_requested='sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&time='+str(time_interval)
# metrics['CPU_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)

# expr_pod_cpu_rabbitmq_connector = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector)
# metrics['Memory_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_memory_rabbitmq_connector)

# expr_pod_cpu_rabbitmq_connector_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas", resource="cpu",pod=~"rabbitmq.*"})&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_requested']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector_requested)

# expr_pod_rabbitmq_function = 'sum(kube_pod_container_status_ready{namespace="openfaas",pod=~"rabbitmq.*"})&time='+str(time_interval)
# metrics['pod_count_rabbitmq']=getdataprometheus(pre_url+expr_pod_rabbitmq_function)

# expr_pod_aeneas_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"aeneas.*"})&time='+str(time_interval)
# expr_pod_tocloud_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"scaling-aeneas-tocloud.*"})&time='+str(time_interval)
# expr_pod_getobject_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"getobject.*"})&time='+str(time_interval)
# print(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_aeneas']=getdataprometheus(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_tocloud']=getdataprometheus(pre_url+expr_pod_tocloud_function)
# metrics['pod_count_getobject']=getdataprometheus(pre_url+expr_pod_getobject_function)

# metrics

In [419]:
promo_metrics

,cpu_used,cpu_requested,time,pod_count,memory_used,seconds,throughput
0,0.025094386973676826,4,2023-01-17 10:37:04,4,521810.01140450884,0,0
1,0.05668776656928747,2.5,2023-01-17 10:37:14,4,2942188.828242379,10,0.26666666666666666
2,0.23230187867081753,2.5,2023-01-17 10:37:24,4,5543045.620818839,20,0.4666666666666667
3,0.5107962938436139,2.5,2023-01-17 10:37:34,4,8182129.923889462,30,0.7333333333333334
4,0.6045528565690699,2.5,2023-01-17 10:37:44,4,6476844.13090238,40,1
5,0.731277911556758,2.5,2023-01-17 10:37:54,4,5934186.92658874,50,1
6,1.7335613779824959,2.5,2023-01-17 10:38:04,4,11518933.686872317,60,1.3333333333333333
7,2.044949349028767,5.5,2023-01-17 10:38:14,4,11629066.605041808,70,1.1333333333333333
8,1.119239823431222,5.5,2023-01-17 10:38:24,4,5130483.630425328,80,1.4
9,2.756499302009331,5.5,2023-01-17 10:38:34,4,2662096.6980612823,90,2.0666666666666664


In [420]:
df_metrics = pd.read_csv("overall_rps.csv")
df_metrics = df_metrics.append(metrics, ignore_index=True)

/tmp/ipykernel_17818/2105317982.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metrics = df_metrics.append(metrics, ignore_index=True)


In [421]:
df_metrics.to_csv("overall_rps.csv",index=False)
display(df_metrics)

,Average Workload TPT,Workload TPT,workload_size,workload_type,Average FET
0,13.343982,142.634278,37,37_120_rps_0.04,13.222224
1,21.712464,163.892761,45,45_120_rps_0.05,21.366818
2,23.109168,181.856853,50,50_120_rps_0.06,25.198708
3,23.998150,167.685005,47,47_120_rps_0.05,23.830660
4,25.439364,173.448969,47,47_120_rps_0.05,24.558274
5,25.439364,173.448969,47,47_120_rps_0.05,24.558274
6,17.387263,155.324578,42,42_120_rps_0.05,16.931921
7,6.552397,123.298365,45,45_120_rps_0.05,6.538489


In [422]:
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
 #   data=res.get('data').get('result')[0].get('value')[1]
    return res

In [423]:

# Frequency
stats_df = df \
.groupby('PT') \
['PT'] \
.agg('count') \
.pipe(pd.DataFrame) \
.rename(columns = {'PT': 'frequency'})

# PDF
stats_df['pdf'] = stats_df['frequency'] / sum(stats_df['frequency'])
stats_df['cdf'] = stats_df['pdf'].cumsum()
stats_df = stats_df.reset_index()
display(stats_df)


stats_df.to_csv("stats_df_"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')
# from matplotlib import pyplot as plt

# plt.plot(stats_df['PT'],stats_df['cdf'],  linestyle='--', marker='o', color='b', label='k8s_k8s')
# #plt.plot(df['user'],df['CT'],  linestyle='--', marker='*', color='r', label='Communication Time')
# plt.grid()
# plt.legend(loc='best')
# plt.xlabel('PT (Seconds)', fontsize=18)
# plt.ylabel('CDF', fontsize=16)


,PT,frequency,pdf,cdf
0,4.0,7,0.155556,0.155556
1,5.0,13,0.288889,0.444444
2,6.0,4,0.088889,0.533333
3,7.0,9,0.200000,0.733333
4,8.0,5,0.111111,0.844444
5,9.0,2,0.044444,0.888889
6,10.0,2,0.044444,0.933333
7,11.0,1,0.022222,0.955556
8,12.0,2,0.044444,1.000000
